In [11]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras import Input

dataset_directory = r"C:\Users\asus\Downloads\homer_bart"

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_directory,
    image_size=(64, 64),  
    label_mode="binary"   
)

train_dataset = dataset.take(int(len(dataset)*0.8))
test_dataset = dataset.skip(int(len(dataset)*0.8))

print(f'Train dataset batches: {len(train_dataset)}')
print(f'Test dataset batches: {len(test_dataset)}')

model = tf.keras.Sequential()
model.add(Input((64,64,3)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_dataset,
          epochs = 20,
          batch_size = 32,
          verbose=1,
          validation_data = test_dataset)

Found 269 files belonging to 2 classes.
Train dataset batches: 7
Test dataset batches: 2
Epoch 1/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 113ms/step - accuracy: 0.4311 - loss: 711.1868 - val_accuracy: 0.7556 - val_loss: 9.1795
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.5962 - loss: 163.5351 - val_accuracy: 0.6444 - val_loss: 529.5518
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.5429 - loss: 405.3706 - val_accuracy: 0.7111 - val_loss: 100.0585
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.6321 - loss: 104.0480 - val_accuracy: 0.8222 - val_loss: 28.4148
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.7397 - loss: 41.6745 - val_accuracy: 0.6667 - val_loss: 39.1463
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.7454 - loss: 48.2464 - val_accuracy: 0.7778 - val_loss: 30.8721
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.7597 - loss: 34.9874 - val_accuracy: 0.6000 - val_loss: 51.0881
Epoch 8/20
7/7 ━

In [4]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

def load_images(directory, target_size=(64, 64)):
    images = []
    labels = []
    for class_name in os.listdir(directory):
        class_dir = os.path.join(directory, class_name)
        if os.path.isdir(class_dir):
            for file_name in os.listdir(class_dir):
                file_path = os.path.join(class_dir, file_name)
                if file_path.endswith('.bmp'):
                    image = Image.open(file_path).convert('RGB')
                    image = image.resize(target_size)
                    image = np.array(image)
                    images.append(image)
                    if file_name.startswith('b'):
                        labels.append(0)  # 0 for Bart
                    elif file_name.startswith('h'):
                        labels.append(1)  # 1 for Homer
    return np.array(images), np.array(labels)

# Load the dataset
images, labels = load_images(r"C:\Users\asus\Downloads\homer_bart")

# Normalize the images
images = images / 255.0

# Create data generators
train_datagen = ImageDataGenerator(  rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.15,
    fill_mode='nearest')
test_datagen = ImageDataGenerator()

# Build the model
def build_model(input_shape=(64, 64, 3)):
    model = Sequential([
        Flatten(input_shape=input_shape),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    optimizer = Adam(learning_rate=0.00001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Initialize stratified K-fold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

accuracies = []

for train_index, test_index in skf.split(images, labels):
    X_train, X_test = images[train_index], images[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
    test_generator = test_datagen.flow(X_test, y_test, batch_size=32)

    model = build_model()

    model.fit(
        train_generator,
        epochs=30,
        callbacks=[early_stopping]
    )

    test_loss, test_accuracy = model.evaluate(test_generator)
    accuracies.append(test_accuracy)

    print(f'Fold Test Accuracy: {test_accuracy}')

print(f'Median Test Accuracy: {np.median(accuracies)}')


Epoch 1/30


C:\Users\asus\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
C:\Users\asus\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.4734 - loss: 0.7856
Epoch 2/30


C:\Users\asus\anaconda3\Lib\site-packages\keras\src\callbacks\early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.5934 - loss: 0.6935
Epoch 3/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.6254 - loss: 0.6719
Epoch 4/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.6397 - loss: 0.6269
Epoch 5/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.6661 - loss: 0.6239
Epoch 6/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.6657 - loss: 0.6230
Epoch 7/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.6690 - loss: 0.5841
Epoch 8/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.7335 - loss: 0.5989
Epoch 9/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.6846 - loss: 0.6072
Epoch 10/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.7472 - loss: 0.5841
Epoch 11/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.7228 - loss: 0.5829
Epoch 12/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.7034 - loss: 0.5641
Epoch 13/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.6871 - loss: 0.6013
Epoch 14/30